<a href="https://colab.research.google.com/github/7201krap/PYTORCH_project/blob/main/sparsity_neuron_size.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from torchvision import transforms
import torchvision.datasets as datasets
import matplotlib.pyplot as plt
import numpy as np
import pickle
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
torch.manual_seed(1234)
np.random.seed(1234)

In [ ]:
mnist_trainset = datasets.MNIST(root='./data', train=True, 
                                download=True, 
                                transform=transforms.Compose([transforms.ToTensor()]))

mnist_testset  = datasets.MNIST(root='./data', 
                                train=False, 
                                download=True, 
                                transform=transforms.Compose([transforms.ToTensor()]))

train_dataloader = torch.utils.data.DataLoader(mnist_trainset, 
                                               batch_size=50, 
                                               shuffle=True)

test_dataloader  = torch.utils.data.DataLoader(mnist_testset, 
                                               batch_size=50, 
                                               shuffle=False)

print("Training dataset size: ", len(mnist_trainset))
print("Testing dataset size: ",  len(mnist_testset))

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw
Processing...
Done!
Training dataset size:  60000
Testing dataset size:  10000


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:480: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [ ]:
# Define the model 
class Model(torch.nn.Module):
    def __init__(self, hidden_layer_neurons):
        super(Model, self).__init__()
        self.linear_1 = torch.nn.Linear(784, hidden_layer_neurons)
        self.linear_2 = torch.nn.Linear(hidden_layer_neurons, 10)
        self.sigmoid  = torch.nn.Sigmoid()

    def forward(self, x):
        x = x.reshape(x.size(0), -1)
        x = self.linear_1(x)
        x = self.sigmoid(x)
        pred = self.linear_2(x)

        return pred

In [ ]:
def get_activation(model):    
    def hook(module, input, output):
        model.layer_activations = output
    return hook

In [ ]:
no_epochs = 30
criterion = torch.nn.CrossEntropyLoss()
def sparsity_trainer(optimizer, model, hidden_layer_neurons):

    # reset the model 
    train_loss = list()
    test_loss  = list()
    test_acc   = list()
    final_spareness = list()

    # define activation list 

    best_test_loss = 1

    for epoch in range(no_epochs):
        total_train_loss = 0
        total_test_loss = 0

        hidden_layer_activation_list = list()

        # training
        # set up training mode 
        model.train()
        for itr, (image, label) in enumerate(train_dataloader):

            optimizer.zero_grad()

            pred = model(image)

            loss = criterion(pred, label)
            total_train_loss += loss.item()

            loss.backward()
            optimizer.step()

        total_train_loss = total_train_loss / (itr + 1)
        train_loss.append(total_train_loss)

        # testing 
        # change to evaluation mode 
        model.eval()
        total = 0
        for itr, (image, label) in enumerate(test_dataloader):

            pred = model(image)

            loss = criterion(pred, label)
            total_test_loss += loss.item()

            # we now need softmax because we are testing.
            pred = torch.nn.functional.softmax(pred, dim=1)
            for i, p in enumerate(pred):
                if label[i] == torch.max(p.data, 0)[1]:
                    total = total + 1

            hidden_layer_activation_list.append(model.layer_activations)

        # this conains activations for all epochs 
        final_spareness.append(hidden_layer_activation_list)

        # caculate accuracy 
        accuracy = total / len(mnist_testset)

        # append accuracy here
        test_acc.append(accuracy)

        # append test loss here 
        total_test_loss = total_test_loss / (itr + 1)
        test_loss.append(total_test_loss)

        print('\nEpoch: {}/{}, Train Loss: {:.8f}, Test Loss: {:.8f}, Test Accuracy: {:.8f}'.format(epoch + 1, no_epochs, total_train_loss, total_test_loss, accuracy))

        # if total_test_loss < best_test_loss:
        #     best_test_loss = total_test_loss
            # print("Saving the model state dictionary for Epoch: {} with Test loss: {:.8f}".format(epoch + 1, total_test_loss))
            # torch.save(model.state_dict(), "model.dth")

    sparseness_list = list()

    for single_epoch_spareness in final_spareness:
        single_epoch_spareness = torch.stack(single_epoch_spareness)
        layer_activations_list = torch.reshape(single_epoch_spareness, (10000, hidden_layer_neurons))

        layer_activations_list = torch.abs(layer_activations_list)  # modified 
        num_neurons = layer_activations_list.shape[1]
        population_sparseness = (np.sqrt(num_neurons) - (torch.sum(layer_activations_list, dim=1) / torch.sqrt(torch.sum(layer_activations_list ** 2, dim=1)))) / (np.sqrt(num_neurons) - 1)
        mean_sparseness_per_epoch = torch.mean(population_sparseness)

        sparseness_list.append(mean_sparseness_per_epoch)

    return test_acc, sparseness_list

In [ ]:
hidden_layer_neurons = [64, 128, 256, 384, 512, 768, 1024]

# AdaDelta

In [ ]:
for i in range(len(hidden_layer_neurons)):
    print(f"Entering {i}-th loop")
    model_Adadelta = Model(hidden_layer_neurons[i])
    print("model_Adadelta:", model_Adadelta)
    model_Adadelta.sigmoid.register_forward_hook(get_activation(model_Adadelta))
    optimizer_Adadelta = torch.optim.Adadelta(model_Adadelta.parameters(), lr=1)
    Adadelta_test_acc, Adadelta_sparseness_list = sparsity_trainer(optimizer=optimizer_Adadelta, model=model_Adadelta, hidden_layer_neurons=hidden_layer_neurons[i])

    if i == 0:
        f = open("neurons_sparsity_Adadelta.txt", "w")
        f.write(str(i)+'\n'+str(Adadelta_test_acc)+'\n'+str(Adadelta_sparseness_list)+'\n\n')
    else:
        f = open("neurons_sparsity_Adadelta.txt", "a")
        f.write(str(i)+'\n'+str(Adadelta_test_acc)+'\n'+str(Adadelta_sparseness_list)+'\n\n')

f.close()

!cp neurons_sparsity_Adadelta.txt /content/drive/MyDrive

Entering 0-th loop
model_Adadelta: Model(
  (linear_1): Linear(in_features=784, out_features=64, bias=True)
  (linear_2): Linear(in_features=64, out_features=10, bias=True)
  (sigmoid): Sigmoid()
)

Epoch: 1/30, Train Loss: 0.45272081, Test Loss: 0.23829514, Test Accuracy: 0.93360000

Epoch: 2/30, Train Loss: 0.20915446, Test Loss: 0.17909380, Test Accuracy: 0.94690000

Epoch: 3/30, Train Loss: 0.16421469, Test Loss: 0.14749706, Test Accuracy: 0.95850000

Epoch: 4/30, Train Loss: 0.13715678, Test Loss: 0.13100217, Test Accuracy: 0.96050000

Epoch: 5/30, Train Loss: 0.11852797, Test Loss: 0.11825827, Test Accuracy: 0.96600000

Epoch: 6/30, Train Loss: 0.10498592, Test Loss: 0.11073470, Test Accuracy: 0.96720000

Epoch: 7/30, Train Loss: 0.09372254, Test Loss: 0.10677439, Test Accuracy: 0.96720000

Epoch: 8/30, Train Loss: 0.08516297, Test Loss: 0.10170037, Test Accuracy: 0.97040000

Epoch: 9/30, Train Loss: 0.07819226, Test Loss: 0.09489402, Test Accuracy: 0.97190000

Epoch: 10/30, Trai

# AdaGrad

In [ ]:
for i in range(len(hidden_layer_neurons)):
    print(f"Entering {i}-th loop")
    model_Adagrad = Model(hidden_layer_neurons[i])
    print("model_Adagrad:", model_Adagrad)
    model_Adagrad.sigmoid.register_forward_hook(get_activation(model_Adagrad))
    optimizer_Adagrad = torch.optim.Adagrad(model_Adagrad.parameters(), lr=0.1)
    Adagrad_test_acc, Adagrad_sparseness_list = sparsity_trainer(optimizer=optimizer_Adagrad, model=model_Adagrad, hidden_layer_neurons=hidden_layer_neurons[i])

    if i == 0:
        f = open("neurons_sparsity_Adagrad.txt", "w")
        f.write(str(i)+'\n'+str(Adagrad_test_acc)+'\n'+str(Adagrad_sparseness_list)+'\n\n')
    else:
        f = open("neurons_sparsity_Adagrad.txt", "a")
        f.write(str(i)+'\n'+str(Adagrad_test_acc)+'\n'+str(Adagrad_sparseness_list)+'\n\n')

f.close()

!cp neurons_sparsity_Adagrad.txt /content/drive/MyDrive

Entering 0-th loop
model_Adagrad: Model(
  (linear_1): Linear(in_features=784, out_features=64, bias=True)
  (linear_2): Linear(in_features=64, out_features=10, bias=True)
  (sigmoid): Sigmoid()
)

Epoch: 1/30, Train Loss: 0.25470985, Test Loss: 0.15037489, Test Accuracy: 0.95530000

Epoch: 2/30, Train Loss: 0.12868609, Test Loss: 0.12286001, Test Accuracy: 0.96360000

Epoch: 3/30, Train Loss: 0.10050082, Test Loss: 0.11054664, Test Accuracy: 0.96550000

Epoch: 4/30, Train Loss: 0.08345019, Test Loss: 0.10360547, Test Accuracy: 0.96800000

Epoch: 5/30, Train Loss: 0.07192035, Test Loss: 0.09705935, Test Accuracy: 0.96910000

Epoch: 6/30, Train Loss: 0.06383210, Test Loss: 0.09383394, Test Accuracy: 0.97020000

Epoch: 7/30, Train Loss: 0.05750153, Test Loss: 0.09226986, Test Accuracy: 0.97130000

Epoch: 8/30, Train Loss: 0.05210991, Test Loss: 0.08874151, Test Accuracy: 0.97140000

Epoch: 9/30, Train Loss: 0.04764514, Test Loss: 0.08874003, Test Accuracy: 0.97150000

Epoch: 10/30, Train

# SGD

In [ ]:
for i in range(len(hidden_layer_neurons)):
    print(f"Entering {i}-th loop")
    model_SGD = Model(hidden_layer_neurons[i])
    print("model_SGD:", model_SGD)
    model_SGD.sigmoid.register_forward_hook(get_activation(model_SGD))
    optimizer_SGD = torch.optim.SGD(model_SGD.parameters(), lr=0.1)
    SGD_test_acc, SGD_sparseness_list = sparsity_trainer(optimizer=optimizer_SGD, model=model_SGD, hidden_layer_neurons=hidden_layer_neurons[i])

    if i == 0:
        f = open("neurons_sparsity_SGD.txt", "w")
        f.write(str(i)+'\n'+str(SGD_test_acc)+'\n'+str(SGD_sparseness_list)+'\n\n')
    else:
        f = open("neurons_sparsity_SGD.txt", "a")
        f.write(str(i)+'\n'+str(SGD_test_acc)+'\n'+str(SGD_sparseness_list)+'\n\n')

f.close()

!cp neurons_sparsity_SGD.txt /content/drive/MyDrive

Entering 0-th loop
model_SGD: Model(
  (linear_1): Linear(in_features=784, out_features=64, bias=True)
  (linear_2): Linear(in_features=64, out_features=10, bias=True)
  (sigmoid): Sigmoid()
)

Epoch: 1/30, Train Loss: 0.84010790, Test Loss: 0.38418883, Test Accuracy: 0.89820000

Epoch: 2/30, Train Loss: 0.34567706, Test Loss: 0.29593622, Test Accuracy: 0.91760000

Epoch: 3/30, Train Loss: 0.29020889, Test Loss: 0.26605765, Test Accuracy: 0.92460000

Epoch: 4/30, Train Loss: 0.25994722, Test Loss: 0.24099451, Test Accuracy: 0.93120000

Epoch: 5/30, Train Loss: 0.23785265, Test Loss: 0.22121951, Test Accuracy: 0.93870000

Epoch: 6/30, Train Loss: 0.21944984, Test Loss: 0.20620261, Test Accuracy: 0.94170000

Epoch: 7/30, Train Loss: 0.20428450, Test Loss: 0.19488495, Test Accuracy: 0.94470000

Epoch: 8/30, Train Loss: 0.19112998, Test Loss: 0.18264221, Test Accuracy: 0.94770000

Epoch: 9/30, Train Loss: 0.18001741, Test Loss: 0.17621129, Test Accuracy: 0.94720000

Epoch: 10/30, Train Los

# Adam

In [ ]:
for i in range(len(hidden_layer_neurons)):
    print(f"Entering {i}-th loop")
    model_Adam = Model(hidden_layer_neurons[i])
    print("model_Adam:", model_Adam)
    model_Adam.sigmoid.register_forward_hook(get_activation(model_Adam))
    optimizer_Adam = torch.optim.Adam(model_Adam.parameters(), lr=0.001)
    Adam_test_acc, Adam_sparseness_list = sparsity_trainer(optimizer=optimizer_Adam, model=model_Adam, hidden_layer_neurons=hidden_layer_neurons[i])

    if i == 0:
        f = open("neurons_sparsity_Adam.txt", "w")
        f.write(str(i)+'\n'+str(Adam_test_acc)+'\n'+str(Adam_sparseness_list)+'\n\n')
    else:
        f = open("neurons_sparsity_Adam.txt", "a")
        f.write(str(i)+'\n'+str(Adam_test_acc)+'\n'+str(Adam_sparseness_list)+'\n\n')

f.close()

!cp neurons_sparsity_Adam.txt /content/drive/MyDrive

Entering 0-th loop
model_Adam: Model(
  (linear_1): Linear(in_features=784, out_features=64, bias=True)
  (linear_2): Linear(in_features=64, out_features=10, bias=True)
  (sigmoid): Sigmoid()
)

Epoch: 1/30, Train Loss: 0.58806726, Test Loss: 0.28062321, Test Accuracy: 0.92200000

Epoch: 2/30, Train Loss: 0.24281302, Test Loss: 0.20646097, Test Accuracy: 0.94020000

Epoch: 3/30, Train Loss: 0.19095139, Test Loss: 0.17552695, Test Accuracy: 0.94940000

Epoch: 4/30, Train Loss: 0.15998729, Test Loss: 0.15357532, Test Accuracy: 0.95510000

Epoch: 5/30, Train Loss: 0.13784190, Test Loss: 0.13857591, Test Accuracy: 0.96050000

Epoch: 6/30, Train Loss: 0.12040462, Test Loss: 0.12792508, Test Accuracy: 0.96360000

Epoch: 7/30, Train Loss: 0.10663814, Test Loss: 0.11991986, Test Accuracy: 0.96500000

Epoch: 8/30, Train Loss: 0.09543831, Test Loss: 0.11120653, Test Accuracy: 0.96740000

Epoch: 9/30, Train Loss: 0.08585681, Test Loss: 0.10547914, Test Accuracy: 0.97040000

Epoch: 10/30, Train Lo